# LDA
Modelamiento de Tópicos con LDA (Latent Dirichet Allocation) para categorías de la base Barómetro Migrante

In [1]:
#Espacio de trabajo
#!pip install lda2vec
#!pip install git+https://github.com/bmabey/pyLDAvis.git@master#egg=pyLDAvis
#!pip install tensorflow
#!pip install keras
#!pip install -U pip setuptools wheel
#!pip install -U spacy[cuda110,transformers,lookups]
#!python -m spacy download es_core_news_sm
#!python -m spacy download en_core_web_sm
#!python -m spacy download es_dep_news_trf
#!pip install torch===1.7.1+cu110 torchvision===0.8.2+cu110 torchaudio===0.7.2 -f https://download.pytorch.org/whl/torch_stable.html --user
#!pip install cupy-cuda110
#!pip install thinc-gpu-ops
#!python --version
#import lda2vec as l2v
#!pip install gensim

# Procesamiento de texto a través spaCy
Si usted no posee una GPU, no intente correr ninguno de los paquetes que se instalan por !pip. Se le hará una lista de requisitos y formas para correr con exito spaCy desde una GPU con arquitectura Turing:
 1. Instale la versión y el paquete de spaCy en Español para procesar texto en este lenguaje
 2. La versión de CUDA es 11.0, válido para GPUs NVIDIA con versión cuDNN 8.0+. Esto lo necesita su equipo para asignar la memoria a la tarjeta gráfica. Se instala por fuera de Python [CUDA](https://developer.nvidia.com/cuda-downloads).
 3. Instale la versión de Pytorch compatible con su versión de CUDA, recuerde que spaCy tiene transformers, esto tiene que estar especificado para que descargue los paquetes de clasificación
 4. Thinc GPU OPS y CuPy son necesarios para que el equipo reconozca la tarjeta gráfica y la vuelva accesible desde Jupyter.
 
Con esto podrá iniciar a correr modelos complejos en su equipo de trabajo, recuerde que no debe de correr ninguna línea que instale un paquete.

In [2]:
import gensim
from gensim import corpora, similarities, models
from gensim.models.phrases import Phraser
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
try:
    import seaborn
except:
    pass
import pyLDAvis
import pyLDAvis.gensim
import sys
import pandas as pd
import spacy
from nltk.corpus import stopwords
import logging
from gensim.models.wrappers import LdaMallet
from gensim.models import CoherenceModel
from pprint import pprint
import re
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
pyLDAvis.enable_notebook()

C:\Users\JOSE\anaconda3\lib\site-packages\tensorflow\core\framework\tensor_shape_pb2.py:18: DeprecationWarning: Call to deprecated create function FileDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  DESCRIPTOR = _descriptor.FileDescriptor(
C:\Users\JOSE\anaconda3\lib\site-packages\tensorflow\core\framework\tensor_shape_pb2.py:36: DeprecationWarning: Call to deprecated create function FieldDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  _descriptor.FieldDescriptor(
C:\Users\JOSE\anaconda3\lib\site-packages\tensorflow\core\framework\tensor_shape_pb2.py:43: DeprecationWarning: Call to deprecated create function FieldDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  _descriptor.Fiel

In [7]:
df = pd.read_excel('Base_entera.xlsx') # can also index sheet by name or fetch all sheets
df = df.drop(columns = 'ID.1')
df

C:\Users\JOSE\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,ID,Snippet
0,0,Una vez llegué a mi casa y habían 5 venezolano...
1,1,Soy venezolano y es triste admitir esa realida...
2,2,Hice Dulce de lechosa venezolano oyendo y bail...
3,3,@Roberto_MunozC Me refería al mercado interno ...
4,4,"@kenin_nunez @SOMOSRIOCRECID Estos venecos, ha..."
...,...,...
562547,562547,@kikscamp Creo que estamos viendo marchas dife...
562548,562548,"@WRadioColombia Llora por los venezolanos, por..."
562549,562549,Un pastor cristiano fue atacado por un migrant...
562550,562550,@NoticiasRCN Están cagaos porque su guarida ve...


In [8]:
#Eliminamos los URL: Todo lo que empieza por  HTTP
df['Clean_Text']=df['Snippet'].replace(r"http\S+", '', regex=True)
#Eliminamos todos los hashtags
df['Clean_Text']=df['Clean_Text'].replace('#\\w+', '', regex=True)
#Eliminamos los signos de puntuación repetidos dentro de los tweets
df['Clean_Text']=df['Clean_Text'].replace(r'([^0-9a-zA-Z])\1+', r'\1', regex=True)
#Eliminamos los signos de puntuación al inicio de un tweet
df['Clean_Text']=df['Clean_Text'].replace(r'^[^0-9a-zA-Z@#]+', '', regex=True)
#Eliminamos los dobles espacios
df['Clean_Text']=df['Clean_Text'].replace(r'()\1+', r'\1\1', regex=True)
#Eliminamos las letras repetivas dentro de las palabras
df['Clean_Text']=df['Clean_Text'].replace(r'(.)\1+', r'\1\1', regex=True)
# Quitar las comillas de los textos
df['Clean_Text']=df['Clean_Text'].replace('"', '', regex=True)
df=df.reset_index()
df = df.astype({"Clean_Text": str})
df.head(11)

C:\Users\JOSE\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,index,ID,Snippet,Clean_Text
0,0,0,Una vez llegué a mi casa y habían 5 venezolano...,Una vez llegué a mi casa y habían 5 venezolano...
1,1,1,Soy venezolano y es triste admitir esa realida...,Soy venezolano y es triste admitir esa realida...
2,2,2,Hice Dulce de lechosa venezolano oyendo y bail...,Hice Dulce de lechosa venezolano oyendo y bail...
3,3,3,@Roberto_MunozC Me refería al mercado interno ...,@Roberto_MunozC Me refería al mercado interno ...
4,4,4,"@kenin_nunez @SOMOSRIOCRECID Estos venecos, ha...","@kenin_nunez @SOMOSRIOCRECID Estos venecos, ha..."
5,5,5,@penfold_michael De hoy a finales del 2022 ser...,@penfold_michael De hoy a finales del 2022 ser...
6,6,6,#1Feb 🇻🇪 Decenas de migrantes venezolanos se e...,Decenas de migrantes venezolanos se encuentran...
7,7,7,"@BrianSnow007 @anGspot @iKarlyzr Eso es falso,...","@BrianSnow007 @anGspot @iKarlyzr Eso es falso,..."
8,8,8,..mexicano Wade Torres en la guitarra y el col...,mexicano Wade Torres en la guitarra y el colom...
9,9,9,"@fabrorejon Vergacion, ya se puso veneco","@fabrorejon Vergacion, ya se puso veneco"


In [5]:
# Le decimos a spaCy que vamos con toda
#spacy.prefer_gpu()
#spacy.require_gpu()
# Corremos los datos en el core de español
#nlp = spacy.load("es_dep_news_trf", exclude=["ner", "parser"])
nlp = spacy.load("es_core_news_sm", exclude=["ner", "parser"])
#Vemos cuales son los componentes del pipeline
print(nlp.pipe_names)
#Aplicamos el pipe a cada objeto de nuestra lista de tweets
print('---------SPACY-----------')
%time docs = list(nlp.pipe(df.Clean_Text.to_list()))

C:\Users\JOSE\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['tok2vec', 'morphologizer', 'attribute_ruler', 'lemmatizer']
---------SPACY-----------
Wall time: 13min 55s


In [9]:
print('---------TOKENIZATION-----------')
tokens=[]
for doc in docs:
    tokens.append([(token.text) for token in doc])

tokens=np.array(tokens)
tokens=pd.DataFrame(tokens).reset_index()
tokens.columns=['index', 'tokens']
print('---------POS-----------')
pos=[]
for doc in docs:
    pos.append([(token.text, token.pos_) for token in doc])

pos=np.array(pos)
pos=pd.DataFrame(pos).reset_index()
pos.columns=['index', 'pos']
df=pd.merge(df, tokens, how='inner', on='index')
df=pd.merge(df, pos, how='inner', on='index')
print('---------STOPWORDS-----------')
spanish_stopwords=stopwords.words('spanish')
spanish_stopwords.extend(['ser', 'haber'])
df['words']=df['tokens'].map(lambda tokens: [token.lower() for token in tokens if token.isalpha() and token.lower() not in spanish_stopwords and len(token)>1])
#N-GRAMS
tokens = df['words'].tolist()
bigram_model = gensim.models.phrases.Phrases(tokens, min_count=5, threshold=20)
trigram_model = gensim.models.phrases.Phrases(bigram_model[tokens], min_count=2, threshold=20)
tokens = list(trigram_model[bigram_model[tokens]])
print('---------N-GRAMS-----------')
print(trigram_model[bigram_model[tokens[0]]])
print('---------LEMMATIZER-----------')
lemma=[]
for doc in docs:
    lemma.append([(token.lemma_) for token in doc])
lemma=np.array(lemma)
lemma=pd.DataFrame(lemma).reset_index()
lemma.columns=['index', 'lemmatized']
print('---------Datos lemmatized-----------')
print(lemma[:1])

#MERGE LISTS INTO DF
df=pd.merge(df, lemma, how='inner', on='index')
df=df.drop(['index'], axis=1)
print('--------BASE-FINAL--------')
df.head(11)
%time

C:\Users\JOSE\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
C:\Users\JOSE\anaconda3\lib\asyncio\events.py:81: DeprecationWarning: `run_cell_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  self._context.run(self._callback, *self._args)


---------TOKENIZATION-----------


<ipython-input-9-f43683fa4c50>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  tokens=np.array(tokens)


---------POS-----------


<ipython-input-9-f43683fa4c50>:14: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  pos=np.array(pos)


---------STOPWORDS-----------
---------N-GRAMS-----------
['vez', 'llegué', 'casa', 'venezolanos', 'mamá', 'complicidad', 'prestaron', 'casa', 'ducharán', 'veía', 'llevaban', 'días', 'bañarse', 'dieron', 'ropa_comida', 'colchas', 'pudieran', 'dormir']
---------LEMMATIZER-----------


<ipython-input-9-f43683fa4c50>:34: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  lemma=np.array(lemma)


---------Datos lemmatized-----------
   index                                         lemmatized
0      0  [uno, vez, llegar, a, mi, casa, y, haber, 5, v...
--------BASE-FINAL--------
Wall time: 0 ns


In [10]:
dictionary = corpora.Dictionary(tokens)
dictionary.save('tweets.dict');
corpus = [dictionary.doc2bow(text) for text in tokens]
corpora.MmCorpus.serialize('tweets.mm', corpus)
corpora.SvmLightCorpus.serialize('tweets.svmlight', corpus)
corpora.BleiCorpus.serialize('tweets.lda-c', corpus)
corpora.LowCorpus.serialize('tweets.low', corpus)
print('--------CORPUS----------')
print(corpus[:1])
##Then the dictionary and corpus can be used to train using LDA
np.random.seed(123456)
mm = corpora.MmCorpus('tweets.mm')
num_topics=10
%time lda = gensim.models.ldamodel.LdaModel(corpus=mm, id2word=dictionary, num_topics=num_topics, update_every=0, passes=10, alpha=[0.01]*num_topics, eta=[0.01]*len(dictionary.keys()))

C:\Users\JOSE\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
C:\Users\JOSE\anaconda3\lib\asyncio\events.py:81: DeprecationWarning: `run_cell_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  self._context.run(self._callback, *self._args)


--------CORPUS----------
[[(0, 1), (1, 2), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1)]]
Wall time: 28min 39s


In [12]:
[[(dictionary[id], freq) for id, freq in cp] for cp in corpus[:1]]

C:\Users\JOSE\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
C:\Users\JOSE\anaconda3\lib\asyncio\events.py:81: DeprecationWarning: `run_cell_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  self._context.run(self._callback, *self._args)


[[('bañarse', 1),
  ('casa', 2),
  ('colchas', 1),
  ('complicidad', 1),
  ('dieron', 1),
  ('dormir', 1),
  ('ducharán', 1),
  ('días', 1),
  ('llegué', 1),
  ('llevaban', 1),
  ('mamá', 1),
  ('prestaron', 1),
  ('pudieran', 1),
  ('ropa_comida', 1),
  ('venezolanos', 1),
  ('vez', 1),
  ('veía', 1)]]

In [13]:
pprint(lda.print_topics())
doc_lda = lda[corpus]

[(0,
  '0.041*"venezolanos" + 0.023*"venezolano" + 0.011*"país" + 0.009*"si" + '
  '0.006*"hoy" + 0.005*"venezuela" + 0.005*"así" + 0.005*"venezolana" + '
  '0.005*"colombianos" + 0.005*"gente"'),
 (1,
  '0.046*"venezolanos" + 0.011*"venezolana" + 0.011*"colombianos" + '
  '0.009*"colombia" + 0.008*"venezolano" + 0.007*"país" + 0.004*"dos" + '
  '0.004*"así" + 0.003*"venezuela" + 0.003*"ciudad"'),
 (2,
  '0.035*"venezolanos" + 0.026*"venezolano" + 0.009*"país" + 0.009*"si" + '
  '0.006*"migrantes" + 0.005*"venezolana" + 0.004*"colombianos" + '
  '0.004*"colombia" + 0.004*"puede" + 0.004*"personas"'),
 (3,
  '0.036*"venezolanos" + 0.017*"venezolano" + 0.013*"si" + 0.010*"venezolana" '
  '+ 0.007*"venecos" + 0.006*"colombia" + 0.005*"país" + 0.004*"venezuela" + '
  '0.004*"migrantes" + 0.004*"creo"'),
 (4,
  '0.061*"venezolanos" + 0.017*"colombia" + 0.013*"venezolano" + '
  '0.012*"venezuela" + 0.009*"si" + 0.006*"colombianos" + 0.006*"pueblo" + '
  '0.005*"país" + 0.004*"venezolana" + 0

C:\Users\JOSE\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [14]:
vis = pyLDAvis.gensim.prepare(topic_model=lda, corpus=corpus, dictionary=dictionary)
pyLDAvis.display(vis)

C:\Users\JOSE\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Compute Model Perplexity and Coherence Score
Model perplexity and topic coherence provide a convenient measure to judge how good a given topic model is. Topic coherence score, in particular, has been more helpful.

In [20]:
# Compute Perplexity
print('\nPerplexity: ', lda.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda, texts=df['Clean_Text'].to_list(), dictionary=dictionary, coherence='c_v')
with np.errstate(invalid='ignore'):
    coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

C:\Users\JOSE\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Perplexity:  -10.274762865621735

Coherence Score:  nan


# LDA Mallet

In [18]:
mallet_path = r'C:\Users\JOSE\Desktop\Trabajo\BX\No_Supervisado\mallet-2.0.8\bin\mallet' # update this path
import os
os.environ['MALLET_HOME'] = r'C:\Users\JOSE\Desktop\Trabajo\BX\No_Supervisado\mallet-2.0.8'
print('----MALLET----')
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=10, id2word=dictionary)

C:\Users\JOSE\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


----MALLET----


In [22]:
# Convert mallet to gensim type
mallet_model = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(ldamallet)
# Compute Perplexity
print('\nPerplexity: ', mallet_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Show Topics
pprint(ldamallet.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=df['Clean_Text'].to_list(), dictionary=dictionary, coherence='c_v')
with np.errstate(invalid='ignore'):
    coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

C:\Users\JOSE\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
C:\Users\JOSE\anaconda3\lib\site-packages\gensim\models\ldamodel.py:1108: RuntimeWarning: invalid value encountered in multiply
  score += np.sum((self.eta - _lambda) * Elogbeta)
C:\Users\JOSE\anaconda3\lib\site-packages\gensim\models\ldamodel.py:1109: RuntimeWarning: invalid value encountered in subtract
  score += np.sum(gammaln(_lambda) - gammaln(self.eta))



Perplexity:  nan
[(0,
  [('venezolano', 0.08243808767461556),
   ('hizo', 0.005915911537423491),
   ('equipo', 0.005483153456945999),
   ('año', 0.004882876119509479),
   ('colombiano', 0.004460975768649702),
   ('frente', 0.004367909514783575),
   ('partido', 0.004129039463193848),
   ('gran', 0.003942906955461593),
   ('después', 0.0038808627862175083),
   ('tras', 0.0036264816923167603)]),
 (1,
  [('venezolano', 0.06499815432407513),
   ('venezuela', 0.027643591189973584),
   ('pueblo', 0.027300023810018424),
   ('presidente', 0.009581534931214615),
   ('colombiano', 0.009423334137653866),
   ('poder', 0.007758230835731647),
   ('mundo', 0.0073107942478830654),
   ('vamos', 0.006222564546722767),
   ('hermanos', 0.006097921497250662),
   ('quiere', 0.005374033017624208)]),
 (2,
  [('venezolanos', 0.20978668415745627),
   ('colombia', 0.10817368093014453),
   ('país', 0.07580266734541373),
   ('venezuela', 0.02425704934295327),
   ('gobierno', 0.021912435102970498),
   ('millones', 

# Optimal number of topics

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
# Can take a long time to run.
model_list, coherence_values = compute_coherence_values(dictionary=dictionary, corpus=corpus, texts=data_lemmatized, start=2, limit=40, step=6)

In [ ]:
# Show graph
limit=40; start=2; step=6;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [ ]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

In [ ]:
# Select the model and print the topics
optimal_model = model_list[3]
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=10))

# Finding the dominant topic in each sentence

In [ ]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=data)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)

# Find the most representative document for each topic

In [ ]:
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Show
sent_topics_sorteddf_mallet.head()

# Topic distribution across documents

In [ ]:
# Number of Documents for Each Topic
topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()

# Percentage of Documents for Each Topic
topic_contribution = round(topic_counts/topic_counts.sum(), 4)

# Topic Number and Keywords
topic_num_keywords = df_topic_sents_keywords[['Dominant_Topic', 'Topic_Keywords']]

# Concatenate Column wise
df_dominant_topics = pd.concat([topic_num_keywords, topic_counts, topic_contribution], axis=1)

# Change Column names
df_dominant_topics.columns = ['Dominant_Topic', 'Topic_Keywords', 'Num_Documents', 'Perc_Documents']

# Show
df_dominant_topics